In [ ]:
import pickle
import numpy as np
import torch

In [ ]:
# LOADING DATA

dataset = 'assembly'

def open_mapping(path):
    with open(path, 'r') as f:
        mapping = f.readlines()
    mapping = [x.strip() for x in mapping] 
    mapping = [x.split(' ') for x in mapping]
    mapping = [[int(x[0]), x[1]] for x in mapping]
    mapping = [' '.join((str(x[0]),x[1])) for x in mapping]
    return mapping
    
    
if dataset == 'assembly':
    mistake_path = '/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/mistake-annotations/mistake_labels'
    # path_to_pickles = '/home/aleflabo/ego_procedural/OadTR/models/en_3_decoder_5_lr_drop_1/results/'
    # path_to_pickles = '/home/aleflabo/ego_procedural/OadTR/models/edo_model/models/cls_ant/results/'
    # path_to_pickles = '/media/hdd/luca_s/egoProcel_mistakes/data_test/enc_only_text_token_mean_144/'
    # path_to_pickles = '/media/hdd/usr/leo/egoProcel_mistakes/data_test/encoder_only-baseline/'
    path_to_pickles = '/home/aleflabo/ego_procedural/OadTR/models/onlyThis_train+val_en_3_decoder_5_lr_drop_1_512/results/' # Leo's checkpoint
    with open('/home/aleflabo/ego_procedural/OadTR/data/assembly/OadTR_assembly/train+val_cutGT_onlyThis/assembly_test_anno.pickle', 'rb') as outfile:
        test_files_gt = pickle.load(outfile)

    mapping_path = '/home/aleflabo/ego_procedural/OadTR/data/assembly/OadTR_assembly/train+val_cutGT_onlyThis/mapping_noBG_trimmed.txt'
    mapping = open_mapping(mapping_path)
    
elif dataset == 'epicTents':
    path_to_pickles = '/home/aleflabo/ego_procedural/OadTR/models/epicTents/en_3_decoder_5_lr_dr_solved/results/'
    mistake_path = '/home/aleflabo/ego_procedural/OadTR/data/EpicTents/adapted_annotations/mistake_annotations'
    with open('/home/aleflabo/ego_procedural/OadTR/data/EpicTents/OadTR/epicTents_test_anno.pickle', 'rb') as outfile:
        test_files_gt = pickle.load(outfile)
        
    mapping_path = '/home/aleflabo/ego_procedural/OadTR/data/EpicTents/OadTR/mapping.txt'
    mapping = open_mapping(mapping_path)


# loading end frame of the window I'm looking at video
with open(path_to_pickles+'end_frame.pickle', 'rb') as f:
    end = pickle.load(f)
# loading start frame of the window I'm looking at video
with open(path_to_pickles+'start_frame.pickle', 'rb') as f:
    start = pickle.load(f)
# loading video names associated to the frames
with open(path_to_pickles+'video_names_test.pickle', 'rb') as f:
    video_names = pickle.load(f)
# loading encoder embedddings after softmax
with open(path_to_pickles+'results_enc_test.pickle', 'rb') as f:
    res_enc = pickle.load(f)
# # loading decoder embedddings after softmax
# with open(path_to_pickles+'results_dec_test_7.pickle', 'rb') as f:
#     res_dec = pickle.load(f)

start = np.array(start)
end = np.array(end)
res_enc = np.asarray(res_enc).T
# res_dec = np.asarray(res_dec['probs'])
# res_enc = np.asarray(all_probs["cls"]).T
# res_dec = np.asarray(all_probs["ant"]).T


In [ ]:
def take_most_frequent(actual, attention_pred):
    # actual = seg_preds[indices]
    temp = np.zeros_like(actual)
    window = 200
    for start in range(0, actual.shape[0], window):
        
        end = start+window

        # counts = np.bincount(actual[start:end])
        counts = np.bincount(attention_pred[start:end].reshape(-1))
        temp[start:end] = np.argmax(counts)
    return temp

def compute_durations(label_list):
    durations = []
    actual = label_list[0]
    count = 0
    length = 0
    for label in label_list:
        if label == actual:
            count += 1
        else:
            durations.append((actual, count))
            length += count
            count = 1
            actual = label
    durations.append((int(actual), int(count)))
    length += count
    print(length)

    return durations

In [ ]:
# preparing json for Edo
dict_for_edo = {}
for video in set(video_names):
    print(video)
    indices = np.where(np.array(video_names) == video)[0]
    repeat = False
    if repeat:
        res = torch.tensor(res_enc.T[indices]).repeat_interleave(5, axis=0).numpy()
    else:
        res = res_enc.T[indices]
    attention_pred = np.argsort(res, axis=1)[:, -1:]
    seg_preds = torch.softmax(torch.tensor(res).permute(1,0), dim=0).argmax(0).numpy()
    gt_durations = compute_durations(torch.softmax(torch.tensor(test_files_gt[video]['anno']).permute(1,0), dim=0).argmax(0).numpy())
    pred_durations = compute_durations(take_most_frequent(seg_preds, attention_pred))

    gt_durations, pred_durations

    dict_for_edo[video] = {}
    dict_for_edo[video]['gt'] = [int(x[0]) for x in gt_durations]
    dict_for_edo[video]['pred'] =  [int(x[0]) for x in pred_durations]
    dict_for_edo[video]['gt_durations'] = [int(x[1]) for x in gt_durations]
    dict_for_edo[video]['pred_durations'] =  [int(x[1]) for x in pred_durations]


# with open(path_to_pickles+'edo.json', 'w') as outfile:
#     json.dump(dict_for_edo, outfile)

In [2]:
import pickle

data = pickle.load(open('/media/hdd/usr/leo/MiniROAD/output_miniROAD.pickle', 'rb'))

In [5]:
data['nusar-2021_action_both_9044-a08_9044_user_id_2021-02-05_154403']

{'anno': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'feature_length': 2503}